In [3]:
from discopy.pivotal import Ty, Box, Diagram
from discopy.pregroup import Word

s, n = Ty('s'), Ty('n')
Alice = Word('Alice', n)
loves = Word('loves', n.r @ s @ n.l)
Bob =  Word('Bob', n)
who = Word('who', n.r @ n @ s.l @ n)
is_rich = Word('is rich', n.r @ s)

vocab = [Alice, who, is_rich, loves, Bob]

love_box = Box('love_box', n @ n, s)
is_rich_box = Box('is_rich_box', n, s)
split = Box('split', n, n @ n)
update = Box('update', n @ s, n)

In [ ]:
from discopy.pivotal import PivotalFunctor

ob = {n: n, s: s}
ar = {Alice: Alice,
      Bob: Bob,
      loves: Cap(n.r, n) @ Cap(n, n.l) >> Diagram.id(n.r) @ love_box @ Diagram.id(n.l),
      is_rich: Cap(n.r, n) >> Diagram.id(n.r) @ is_rich_box,
      who: Cap(n.r, n) >> Diagram.id(n.r) @ (split >> Diagram.id(n) @ Cap(s, s.l) @ Diagram.id(n) >>
                                             update @ Diagram.id(s.l @ n))
     }

F = PivotalFunctor(ob, ar, ob_cls=Ty, ar_cls=Diagram)

In [ ]:
F(who).draw()

In [ ]:
from time import time
from discopy.disco import brute_force

gen, n_sentences = brute_force(*vocab), 20
sentences, parsing = list(), dict()

print("Brute force search for grammatical sentences:")

start = time()
for i in range(n_sentences):
    diagram = next(gen)
    sentence = ' '.join(str(w)
        for w in diagram.boxes if isinstance(w, Word)) + '.'
    sentences.append(sentence)
    parsing.update({sentence: diagram})
    print(sentence)

print("{:.2f} seconds to generate {} sentences.\n".format(time() - start, n_sentences))

In [ ]:
print(sentences[19])
parsing[sentences[19]].draw()

In [ ]:
F(parsing[sentences[19]]).normal_form().draw()

In [ ]:
def has_cups(diagram):
    for box in diagram.boxes:
        if isinstance(box, Cup) or isinstance(box, Cap):
            return True
    return False

In [ ]:
autonomised_sentences = [F(parsing[sentences[i]]).normal_form() for i in range(n_sentences)]

In [ ]:
for i in range(n_sentences):
    assert not has_cups(autonomised_sentences[i])

In [ ]:
autonomised_sentences[6].draw()